# Generate primers for spike mutations in GISAID alignments

This script takes `spike_alignment_counts.csv` table, filters it for amino acids that are not already present in variant sequence and runs `create_primers_del.py` script to generate primers.

In [ ]:
import pandas as pd
from plotnine import *
from Bio.SeqUtils import MeltingTemp as mt
import statistics

In [ ]:
# import snakemake variables
gisaid_mutations = snakemake.input.gisaid_mutations
reference_lookup  = snakemake.input.reference_lookup
codon_table = snakemake.input.codon_table
spike_extended = snakemake.input.spike_extended
create_primers_srcipt = snakemake.input.create_primers_srcipt
gisaid_mutation_count_filter = snakemake.params.gisaid_mutation_count_filter
deletion_count_filter = snakemake.params.deletion_count_filter

variant_gisaid_mutations_out = snakemake.output.variant_gisaid_mutations
variant_gisaid_primers_del = snakemake.output.variant_gisaid_primers_del
variant_gisaid_primers = snakemake.output.variant_gisaid_primers

In [ ]:
# gisaid_mutations = '../results/spike_alignment_counts.csv'
# reference_lookup  = '../reference_sequences/reference_sequence_position_lookup.csv'
# codon_table = '../reference_sequences/homo_codon_freq_del.csv'
# spike_extended = '../reference_sequences/Omicron_BA.1_extended_ends_for_primers.txt'
# create_primers_srcipt = '../scripts/create_primers_del.py'
# gisaid_mutation_count_filter = 16

# variant_gisaid_mutations_out = '../results/variant_gisaid_mutations.csv'
# variant_gisaid_primers_del = '../results/variant_gisaid_primers_del.csv'
# variant_gisaid_primers = '../results/pool_variant_gisaid_mutation_primers.csv'
# deletion_count_filter = 200


In [ ]:
gisaid_mutations = pd.read_csv(gisaid_mutations)
reference_lookup = pd.read_csv(reference_lookup)

In [ ]:
#rename columns
gisaid_mutations = gisaid_mutations.drop(columns=['wildtype'])
gisaid_mutations = gisaid_mutations.rename(columns={"mutant": "amino_acid", "count": "alignment_counts"}).reset_index(drop=True)
# change deletion character
gisaid_mutations['amino_acid'] = gisaid_mutations['amino_acid'].str.replace('del','-')
#remove any mutations that are not single substitutions or deletions
gisaid_mutations = gisaid_mutations[gisaid_mutations['amino_acid'].str.len() == 1]


In [ ]:
#remove deletions outside NTD region
gisaid_mutations = gisaid_mutations[(gisaid_mutations.amino_acid != '-') | (gisaid_mutations.site <= 303 )]

In [ ]:
gisaid_mutations

In [ ]:
#plot deletion counts
p = (ggplot(gisaid_mutations[(gisaid_mutations.amino_acid == '-') ]) + 
     aes('alignment_counts') + 
     geom_histogram(bins=500)+
     xlim(0,1000)
    )
p.draw

In [ ]:
#plot deletion counts along spike NTD
p = (ggplot(gisaid_mutations[(gisaid_mutations.amino_acid == '-') ]) + 
     aes('site', 'alignment_counts') + 
     geom_point()+
     xlim(0,303)+
     scale_y_continuous(trans='log10')
    )
p.draw

In [ ]:
#Filter out NTD deletions
gisaid_mutations = gisaid_mutations[(gisaid_mutations.amino_acid == '-') & (gisaid_mutations.alignment_counts > deletion_count_filter ) | (gisaid_mutations.amino_acid != '-')]


In [ ]:
#plot selected deletion counts along spike NTD
print(len(gisaid_mutations[(gisaid_mutations.amino_acid == '-') & (gisaid_mutations.alignment_counts > deletion_count_filter )]))
      
p = (ggplot(gisaid_mutations[(gisaid_mutations.amino_acid == '-')]) + 
     aes('site', 'alignment_counts') + 
     geom_point()+
     xlim(0,303)+
     scale_y_continuous(trans='log10')
    )
p.draw

In [ ]:
#Merge any duplicated sites
gisaid_mutations = gisaid_mutations.groupby(['site','amino_acid']).agg({'alignment_counts': 'sum'})
gisaid_mutations.reset_index(inplace=True)

In [ ]:
#plot alignment_counts for nondeletions
p = (ggplot(gisaid_mutations[(gisaid_mutations.amino_acid != '-') ]) + 
     aes('alignment_counts') + 
     geom_histogram(bins=500)+
     xlim(0,500)
    )
p.draw

In [ ]:
#remove GISAID mutations that are in variant spike already
#merge lookup and gisaid tables
new_df = pd.merge(gisaid_mutations,
                  reference_lookup,
                  how='left',
                  left_on=['site','amino_acid'],
                  right_on = ['parent_pos','parent_seq'],
                  indicator=True)

#filter on amino acids not already present in variant
new_df_noWU = new_df.loc[(new_df['_merge'] == 'left_only') | (new_df['variant_sig'] == 'Yes')]
new_gisaid_mutations = new_df[new_df.index.isin(new_df_noWU.index)]
new_gisaid_mutations = new_gisaid_mutations[['site','amino_acid','alignment_counts']]

In [ ]:
#now merge tables just on parent position, sort by position, and renumber by variant
new_gisaid_mutations = pd.merge(new_gisaid_mutations,
                  reference_lookup,
                  how='left',
                  left_on=['site'],
                  right_on = ['parent_pos'])

#drop positions with NaN in parent sequence removed CTD amino acids
new_gisaid_mutations = new_gisaid_mutations[new_gisaid_mutations['parent_pos'].notna()]
new_gisaid_mutations = new_gisaid_mutations.sort_values(by=['site'],ignore_index=True )

In [ ]:
#plot alignment_counts for nondeletions after filtering
print('number of sites:', len(gisaid_mutations[(gisaid_mutations.amino_acid != '-') & (gisaid_mutations.alignment_counts >= gisaid_mutation_count_filter)].index))
p = (ggplot(gisaid_mutations[(gisaid_mutations.amino_acid != '-') & (gisaid_mutations.alignment_counts >= gisaid_mutation_count_filter)]) + 
     aes('alignment_counts') + 
     geom_histogram(bins=500)+
     xlim(0,500)
    )
p.draw


In [ ]:
#drop mutations that occur only once
new_gisaid_mutations = new_gisaid_mutations.loc[new_gisaid_mutations['alignment_counts'] > gisaid_mutation_count_filter]


In [ ]:
#create new table with mutations according to variant numbering
variant_gisaid_mutations = new_gisaid_mutations[['variant_pos', 'amino_acid', 'alignment_counts']].copy()
variant_gisaid_mutations = variant_gisaid_mutations.rename(
                            columns={"variant_pos": "site",
                                     "amino_acid": "mutant",
                                     "alignment_counts": "alignment_counts" }
)
variant_gisaid_mutations.to_csv(variant_gisaid_mutations_out, index=False) 

In [ ]:
# run primer design script
!python {create_primers_srcipt} \
    {spike_extended} \
    {variant_gisaid_mutations_out} \
    {codon_table} \
    variantGISAID \
    {variant_gisaid_primers_del} \
    --minprimertm 60.5 \
    --maxprimertm 61.5

In [ ]:
#import primer table
header_list = ["primer_name", "seq"]
variantGISAID_primers = pd.read_csv(variant_gisaid_primers_del, names=header_list)
variantGISAID_primers

In [ ]:
#strip all --- strings that indicate deletions
variantGISAID_primers['seq'] = variantGISAID_primers['seq'].replace('-', '', regex=True).astype(str)
variantGISAID_primers

## Analyse primers

Analyse primer annealing temperatures.

In [ ]:
variantGISAID_primers['Tm'] = variantGISAID_primers.apply(lambda x: '%0.2f' % mt.Tm_NN(x.seq, strict=False), axis=1)
variantGISAID_primers['Tm'] = variantGISAID_primers['Tm'].astype('float')
variantGISAID_primers['length'] = variantGISAID_primers.apply(lambda x: len(x.seq), axis=1)
variantGISAID_primers

In [ ]:
p = (ggplot(variantGISAID_primers) + 
     aes('length') + 
     geom_histogram(color='white',bins=30)
    )

p.draw

In [ ]:
p = (ggplot(variantGISAID_primers) + 
     aes('Tm') + 
     geom_histogram(color='white', bins=30)+
     xlim(58,72)
    )

p.draw

In [ ]:
statistics.pvariance(variantGISAID_primers['Tm'])

In [ ]:
# finish by exporting primers
variantGISAID_primers.to_csv(variant_gisaid_primers, index=False) 